In [5]:
import pandas as pd
import numpy as np

def daily_select_targets(factortable_path, industry_col="industry_name"):
    """
    每日筛选6个标的：基础池→TOP3行业→行业内TOP2
    参数：
        factortable_path: factortable.parquet的路径
        industry_col: 行业字段名（默认用industry_name，可根据实际字段修改）
    返回：
        包含每日6个标的的DataFrame
    """
    # 1. 加载数据并预处理
    df = pd.read_parquet(factortable_path)
    # 确保日期格式正确，按日期+股票代码排序
    df["date"] = pd.to_datetime(df["date"])
    df = df.sort_values(by=["date", "stock_code"]).reset_index(drop=True)
    
    # 2. 第一层：筛选基础池（竞价量比>1 + 高开）
    basic_pool = df[
        (df["auction_volume_ratio"] > 1)  # 竞价量比大于1（量能提升）
        & (df["is_high_open"] == True)    # 当日高开
        & (df["auction_volume_ratio"].notna())  # 排除空值
        & (df[industry_col].notna())  # 排除无行业信息的标的
    ].copy()
    print(f"基础池每日平均标的数：{basic_pool.groupby('date').size().mean():.0f}（符合200-300预期）")
    
    # 3. 第二层：筛选TOP3行业（按基础池内行业标的数量排序）
    # 按日期+行业分组，统计每个行业的标的数量
    industry_count = basic_pool.groupby(["date", industry_col]).size().reset_index(name="stock_count")
    # 按日期排序，取每个日期下标的数量前3的行业
    industry_count["industry_rank"] = industry_count.groupby("date")["stock_count"].rank(ascending=False, method="dense")
    top3_industries = industry_count[industry_count["industry_rank"] <= 3][["date", industry_col]]
    
    # 4. 第三层：TOP3行业内各选TOP2标的（按竞价量比降序）
    # 只保留TOP3行业的标的
    top3_industry_pool = basic_pool.merge(top3_industries, on=["date", industry_col], how="inner")
    # 每个日期+行业内，按竞价量比降序排，取前2
    top3_industry_pool["stock_rank_in_industry"] = top3_industry_pool.groupby(["date", industry_col])["auction_volume_ratio"].rank(ascending=False, method="dense")
    final_targets = top3_industry_pool[top3_industry_pool["stock_rank_in_industry"] <= 2].copy()
    
    # 5. 整理输出结果（先排序，再筛选输出字段）
    output_cols = ["date", industry_col, "stock_code", "stock_name", 
                   "auction_volume_ratio", "volume_ratio_5d", "auction_rise_ratio"]
    # 补充stock_name（若factortable中没有，需从之前的widetable关联）
    if "stock_name" not in final_targets.columns:
        # 加载widetable并统一date类型（字符串转datetime）
        widetable = pd.read_parquet(r"D:\workspace\xiaoyao\data\widetable.parquet")
        widetable["date"] = pd.to_datetime(widetable["date"])  # 关键修复：统一date类型
        # 关联股票名称
        final_targets = final_targets.merge(
            widetable[["date", "stock_code", "stock_name"]], 
            on=["date", "stock_code"], 
            how="left"
        )
    
    # 确保输出字段都存在（避免因关联失败导致stock_name缺失）
    final_targets = final_targets.dropna(subset=["stock_name"])
    # 关键修复：先按指定字段排序，再筛选output_cols（stock_rank_in_industry用于排序，不输出）
    final_targets_sorted = final_targets.sort_values(
        by=["date", industry_col, "stock_rank_in_industry"]
    )[output_cols]
    
    return final_targets_sorted

# --------------------------
# 执行筛选（已适配你的行业字段和路径）
# --------------------------
if __name__ == "__main__":
    targets_df = daily_select_targets(
        factortable_path=r"D:\workspace\xiaoyao\data\factortable.parquet",
        industry_col="jq_l2_industry_name"  # 你的行业字段名
    )
    # 保存结果到本地
    targets_df.to_csv("./daily_6_targets.csv", index=False, encoding="utf-8-sig")
    print(f"筛选完成！共{targets_df['date'].nunique()}个交易日，累计{len(targets_df)}个标的（每日理论6个），结果已保存到daily_6_targets.csv")
    # 打印前10条示例
    print("\n前10条筛选结果示例：")
    print(targets_df.head(10).to_string(index=False))

基础池每日平均标的数：16（符合200-300预期）
筛选完成！共667个交易日，累计9162个标的（每日理论6个），结果已保存到daily_6_targets.csv

前10条筛选结果示例：
      date jq_l2_industry_name  stock_code stock_name  auction_volume_ratio  volume_ratio_5d  auction_rise_ratio
2023-02-06                公路运输 600620.XSHG       天宸股份          2.099738e+00     4.439891e-01            0.002641
2023-02-06                其他电子 002130.XSHE       沃尔核材          1.270565e+00     3.354168e+00            0.016941
2023-02-06                 商用车 600066.XSHG       宇通客车          1.921135e+00     1.036652e+00            0.028747
2023-02-06                园林工程 002310.XSHE       东方园林          1.185617e+00     1.315867e+00            0.004853
2023-02-06            家电零部件及其他 002512.XSHE       达华智能          1.665420e+00     5.060383e+00            0.100181
2023-02-06                工程机械 000157.XSHE       中联重科          4.355474e+00     1.105023e+00            0.008686
2023-02-06                工程机械 600817.XSHG       宇通重工          1.957298e+00     1.994528e+00            0.09970

In [ ]:
import pandas as pd
import numpy as np

def calculate_returns_and_cumulative():
    """
    核心修复：基于widetable实际交易日历，匹配每个标的T日后的第一个交易日（真实T+1日）
    步骤：1. 加载数据 → 2. 构建股票交易日历 → 3. 匹配T日/T+1日价格 → 4. 计算收益 → 5. 统计累积收益
    """
    # --------------------------
    # 1. 加载基础数据
    # --------------------------
    # 加载筛选出的每日标的（T日标的）
    targets_df = pd.read_csv("./daily_6_targets.csv", encoding="utf-8-sig")
    targets_df["date"] = pd.to_datetime(targets_df["date"])  # T日（datetime类型）
    print(f"加载筛选标的：{targets_df['date'].nunique()}个交易日，{len(targets_df)}个标的")
    
    # 加载widetable，提取所有股票的实际交易日历（只保留股票代码、日期、开盘价、收盘价）
    widetable = pd.read_parquet(r"D:\workspace\xiaoyao\data\widetable.parquet")
    widetable["date"] = pd.to_datetime(widetable["date"])  # 统一为datetime类型
    price_df = widetable[["stock_code", "date", "open", "close"]].copy()
    # 去重（确保每个股票-日期只有一条记录）
    price_df = price_df.drop_duplicates(subset=["stock_code", "date"], keep="first")
    # 按股票代码+日期排序（为后续找下一个交易日做准备）
    price_df = price_df.sort_values(by=["stock_code", "date"]).reset_index(drop=True)
    print(f"加载股票交易日历：{price_df['stock_code'].nunique()}只股票，{price_df['date'].nunique()}个实际交易日")
    
    # --------------------------
    # 2. 构建T+1日映射表（核心修复：找每个股票-T日的下一个交易日）
    # --------------------------
    def get_next_trading_day(group):
        """按股票分组，给每个交易日匹配下一个交易日（shift(-1)取后一行）"""
        # 新增T+1日日期列（下一个交易日）
        group["t1_date"] = group["date"].shift(-1)
        # 新增T+1日开盘价、收盘价（下一个交易日的价格）
        group["t1_open"] = group["open"].shift(-1)
        group["t1_close"] = group["close"].shift(-1)
        return group
    
    # 按股票分组，生成每个股票的“交易日→下一个交易日”映射
    price_with_t1 = price_df.groupby("stock_code", group_keys=False).apply(get_next_trading_day)
    # 过滤掉无下一个交易日的记录（比如最新的交易日，没有T+1日）
    price_with_t1 = price_with_t1.dropna(subset=["t1_date"])
    print(f"构建T+1日映射表：覆盖{price_with_t1['stock_code'].nunique()}只股票，{price_with_t1['date'].nunique()}个交易日")
    
    # --------------------------
    # 3. 关联T日和真实T+1日价格
    # --------------------------
    # 关联T日价格（标的表与价格表按“股票代码+T日”关联）
    targets_t_day = targets_df.merge(
        price_with_t1[["stock_code", "date", "open", "close", "t1_date", "t1_open", "t1_close"]].rename(
            columns={"open": "t_open", "close": "t_close"}  # T日价格重命名
        ),
        on=["stock_code", "date"],  # 精准匹配：同一股票的同一T日
        how="left"
    )
    
    # 过滤无效数据（无T日价格或无T+1日价格的标的）
    targets_full = targets_t_day.dropna(subset=["t_open", "t_close", "t1_date", "t1_open", "t1_close"])
    print(f"关联完成后有效标的：{len(targets_full)}个（过滤无T日/T+1日价格的记录）")
    print(f"真实T+1日示例：T日{targets_full['date'].iloc[0].strftime('%Y-%m-%d')} → T+1日{targets_full['t1_date'].iloc[0].strftime('%Y-%m-%d')}")
    
    # --------------------------
    # 4. 计算4类收益率（逻辑不变，基于真实T+1日）
    # --------------------------
    def safe_divide(a, b):
        """安全除法：避免除以0"""
        return a / b.replace(0, 0.0001)
    
    # 4类收益率（基于真实T+1日价格）
    targets_full["return_t1open_topen"] = safe_divide(targets_full["t1_open"] - targets_full["t_open"], targets_full["t_open"])
    targets_full["return_t1open_tclose"] = safe_divide(targets_full["t1_open"] - targets_full["t_close"], targets_full["t_close"])
    targets_full["return_t1close_topen"] = safe_divide(targets_full["t1_close"] - targets_full["t_open"], targets_full["t_open"])
    targets_full["return_t1close_tclose"] = safe_divide(targets_full["t1_close"] - targets_full["t_close"], targets_full["t_close"])
    
    # --------------------------
    # 5. 计算每日平均收益（按T日分组，而非T+1日）
    # --------------------------
    daily_return_cols = [
        "return_t1open_topen", 
        "return_t1open_tclose", 
        "return_t1close_topen", 
        "return_t1close_tclose"
    ]
    # 按T日分组（因为策略是T日选股，收益对应T日的决策）
    daily_avg_returns = targets_full.groupby("date")[daily_return_cols].mean().reset_index()
    daily_avg_returns.columns = ["date"] + [f"avg_{col}" for col in daily_return_cols]
    print(f"计算每日平均收益：{len(daily_avg_returns)}个有效T日")
    
    # --------------------------
    # 6. 计算累积收益（按T日时间顺序连乘）
    # --------------------------
    # 选择核心收益指标（默认T+1日收盘相对T日收盘的平均收益，贴近实际持有收益）
    core_return_col = "avg_return_t1close_tclose"
    # 按T日排序（确保时间顺序正确）
    daily_avg_returns = daily_avg_returns.sort_values("date").reset_index(drop=True)
    # 计算每日累积因子：1 + 0.5*每日平均收益
    daily_avg_returns["cumulative_factor"] = 1 + 0.5 * daily_avg_returns[core_return_col]
    # 连乘得到累积收益（初始值1，即首日前的收益为1）
    daily_avg_returns["cumulative_return"] = daily_avg_returns["cumulative_factor"].cumprod()
    # 累积收益率（相对初始值的涨幅）
    daily_avg_returns["cumulative_return_rate"] = daily_avg_returns["cumulative_return"] - 1
    
    # --------------------------
    # 7. 保存结果（3个核心文件）
    # --------------------------
    # 1. 标的级详细收益（含T+1日真实日期）
    targets_full.to_csv("./targets_detailed_returns_fixed.csv", index=False, encoding="utf-8-sig")
    # 2. 每日平均收益
    daily_avg_returns[["date"] + [f"avg_{col}" for col in daily_return_cols]].to_csv("./daily_avg_returns_fixed.csv", index=False, encoding="utf-8-sig")
    # 3. 累积收益（含T日顺序）
    cumulative_cols = ["date", core_return_col, "cumulative_factor", "cumulative_return", "cumulative_return_rate"]
    daily_avg_returns[cumulative_cols].to_csv("./cumulative_returns_fixed.csv", index=False, encoding="utf-8-sig")
    
    # --------------------------
    # 关键统计信息输出
    # --------------------------
    print("\n" + "="*60)
    print("策略收益关键统计（基于真实交易日历）：")
    print(f"回测期间（T日范围）：{daily_avg_returns['date'].min().strftime('%Y-%m-%d')} → {daily_avg_returns['date'].max().strftime('%Y-%m-%d')}")
    print(f"有效T日数：{len(daily_avg_returns)}天")
    print(f"每日平均核心收益（{core_return_col}）：{daily_avg_returns[core_return_col].mean():.4f}（{daily_avg_returns[core_return_col].mean()*100:.2f}%）")
    print(f"累积收益（初始1元）：{daily_avg_returns['cumulative_return'].iloc[-1]:.4f}元")
    print(f"累积收益率：{daily_avg_returns['cumulative_return_rate'].iloc[-1]*100:.2f}%")
    print(f"最大累积收益率：{daily_avg_returns['cumulative_return_rate'].max()*100:.2f}%")
    print(f"最小累积收益率（最大回撤后）：{daily_avg_returns['cumulative_return_rate'].min()*100:.2f}%")
    print("="*60)
    
    return targets_full, daily_avg_returns

# --------------------------
# 执行计算
# --------------------------
if __name__ == "__main__":
    targets_detailed, daily_avg = calculate_returns_and_cumulative()